### Installing Dependencies

In [128]:
import pandas as pd
import numpy as np
import random
from scipy.optimize import minimize

### Preprocessing and Cleaning Data

In [129]:
df = pd.read_csv("dogdata.csv")

# Cleaning:
# Remove rows with NaN values 
df = df.dropna()
# Remove rows that have 'of' as their popularity value
mask = df['popularity'] != 'of'
df = df[mask]

# Preprocessing:
# Calculate average height
df['avg_height'] = (df['min_height'] + df['max_height']) / 2
# Calculate average weight
df['avg_weight'] = (df['min_weight'] + df['max_weight']) / 2
# Calculate max value of max_height
max_height = df['max_height'].max()
# Calculate max value of max_weight
max_weight = df['max_weight'].max()
# Make sure popularity data is in float form
df['popularity'] = df['popularity'].astype(float)
# Calculate max value of popularity
max_pop = df['popularity'].max()
print(max_pop)
# Apply Max Scaling
df['scaled_avg_height'] = df['avg_height'] / max_height
df['scaled_avg_weight'] = df['avg_weight'] / max_weight
df['scaled_pop'] = df['popularity'] / max_pop
# Put Scaled and Relevant data into new dataframe
breed_data = df[['scaled_pop','scaled_avg_height','scaled_avg_weight','grooming_frequency_value','shedding_value','energy_level_value','trainability_value','demeanor_value']]
breed_data = pd.concat([df.iloc[:,0], breed_data], axis=1)
breed_data.columns.values[0] = 'Breeds'

# Save the DataFrame to a new CSV file
breed_data.to_csv('breed_data.csv', index=False)
print("Cleaned data saved to 'breed_data.csv'")

# Repeat this process for the cluster (breed groups) data
df2 = pd.read_csv('merged_data_latest.csv')
# Store the list of breeds in each group in a dictionary so that we can output them to the user
breed_clusters = df2.groupby('Cluster')['Breed'].apply(list).to_dict()
# Preprocessing:
df2 = df2.drop(columns=['Breed', 'popularity'])
df2 = df2.apply(pd.to_numeric, errors='coerce').dropna()
df2 = df2.groupby('Cluster').mean()
df2.head()
# Calculate average height
df2['avg_height'] = (df2['min_height'] + df2['max_height']) / 2
# Calculate average weight
df2['avg_weight'] = (df2['min_weight'] + df2['max_weight']) / 2
# Apply Max Scaling
df2['scaled_avg_height'] = df2['avg_height'] / max_height
df2['scaled_avg_weight'] = df2['avg_weight'] / max_weight
cluster_data = df2[['scaled_avg_height','scaled_avg_weight','grooming_frequency_value','shedding_value','energy_level_value','trainability_value','demeanor_value']]
# Save the DataFrame to a new CSV file
cluster_data.to_csv('cluster_data.csv', index=False)
print("Cleaned data saved to 'cluster_data.csv'")


192.0
Cleaned data saved to 'breed_data.csv'
Cleaned data saved to 'cluster_data.csv'


### Defining function for determining optimal breed combination
1. This first function includes a penalty hyperparameter in the loss function. This hyperparameter would need to be tuned by using some real mixed breed data in order to justify the choice of value for it.

In [104]:
def estimate_dog_breed(breed_data, y):
    # Prepare data for use in loss function
    data = breed_data.drop(columns=['Breeds', 'scaled_pop'])
    B = np.asarray(data).T
    # Extract popularity data to penalise rare breeds in loss function. The hope is that this will give more realistic mixtures.
    pop = breed_data['scaled_pop']
    pop_penalty = 0.05
    # Defining the loss function (SSE) to minimise
    def loss(w):
        x = B @ w
        diffsqrd = (x - y) ** 2
        return diffsqrd.sum() + pop_penalty*(w @ pop) # SSE with pop penalty

    # Defining constraint: Elements of w must sum to 1
    constraint = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

    # Initial guess of weights (Random values between 0 and 1)
    w0 = np.random.uniform(0, 1, size=B.shape[1])
    # Normalise initial guess to meet constraint
    w0 /= np.sum(w0)

    # Define bounds [0,1] for all weights
    bounds = [(0, 1) for _ in range(B.shape[1])]

    # Minimize using SLSQP (supports constraints)
    result = minimize(loss, w0, method='SLSQP', bounds=bounds, constraints=[constraint])

    # Discard weights of less than 0.25
    w_adjusted = np.where(result.x < 0.25, 0, result.x)

    # Renormalize to sum to 1
    if np.sum(w_adjusted) > 0:
        w_adjusted /= np.sum(w_adjusted)  # Normalize to sum to 1

    # Print results
    percents = [round(i*100, 0) for i in w_adjusted if i != 0]
    breeds = [breed_data.iloc[i, 0] for i in range(len(w_adjusted)) if w_adjusted[i] != 0]
    print('We estimate your dog to be:')
    for i in range(len(breeds)):
        print(percents[i], '% ', breeds[i])

2. Here, instead of using a penalty and hyperparameter to deal with the issue of returning highly improbable breed combinations, we will simply only use the 50 most popular breeds in our dataset. To make up for the oversight of rarer dog breeds and offer the user with some useful information we will, aswell as proposing a breed mixture, return a group/s of dogs that their dog is most similar to. This will be done by using the same optimal weights technique.

In [114]:
def estimate_dog_breed_pop(breed_data, y):
    # Extract only the 50 most popular dog breeds
    scaled_val = 50 / max_pop
    most_pop = breed_data['scaled_pop'] <= scaled_val
    pop_breed_data = breed_data[most_pop]
    pop_data = pop_breed_data.drop(columns=['Breeds', 'scaled_pop'])
    Bpop = np.asarray(pop_data).T
    # Defining the loss function (SSE) to minimise
    def loss(w):
        x = Bpop @ w
        diffsqrd = (x - y) ** 2
        return diffsqrd.sum() # SSE

    # Defining constraint: Elements of w must sum to 1
    constraint = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

    # Initial guess of weights (Random values between 0 and 1)
    w0 = np.random.uniform(0, 1, size=Bpop.shape[1])
    # Normalise initial guess to meet constraint
    w0 /= np.sum(w0)

    # Define bounds [0,1] for all weights
    bounds = [(0, 1) for _ in range(Bpop.shape[1])]

    # Minimize using SLSQP (supports constraints)
    result = minimize(loss, w0, method='SLSQP', bounds=bounds, constraints=[constraint])

    # Discard weights of less than 0.25
    w_adjusted = np.where(result.x < 0.25, 0, result.x)

    # Renormalize to sum to 1
    if np.sum(w_adjusted) > 0:
        w_adjusted /= np.sum(w_adjusted)  # Normalize to sum to 1

    # Print results
    percents = [round(i*100, 0) for i in w_adjusted if i != 0]
    breeds = [pop_breed_data.iloc[i, 0] for i in range(len(w_adjusted)) if w_adjusted[i] != 0]
    print('We estimate your dog to be:')
    for i in range(len(breeds)):
        print(percents[i], '% ', breeds[i])
    estimate_dog_cluster(cluster_data, y)

def estimate_dog_cluster(cluster_data, y):
    cluster_data1 = cluster_data
    C = np.asarray(cluster_data1).T
    # Defining the loss function (SSE) to minimise
    def loss(w):
        x = C @ w
        diffsqrd = (x - y) ** 2
        return diffsqrd.sum() # SSE

    # Defining constraint: Elements of w must sum to 1
    constraint = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

    # Initial guess of weights (Random values between 0 and 1)
    w0 = np.random.uniform(0, 1, size=C.shape[1])
    # Normalise initial guess to meet constraint
    w0 /= np.sum(w0)

    # Define bounds [0,1] for all weights
    bounds = [(0, 1) for _ in range(C.shape[1])]

    # Minimize using SLSQP (supports constraints)
    result = minimize(loss, w0, method='SLSQP', bounds=bounds, constraints=[constraint])

    # Discard weights of less than 0.25
    w_adjusted = np.where(result.x < 0.25, 0, result.x)

    # Renormalize to sum to 1
    if np.sum(w_adjusted) > 0:
        w_adjusted /= np.sum(w_adjusted)  # Normalize to sum to 1

    # Print results
    hierachy = np.argsort(w_adjusted)
    percents = [round(w_adjusted[i]*100, 0) for i in hierachy if w_adjusted[i] != 0]
    groups = [i for i in hierachy if w_adjusted[i] != 0]
    print("You're dog fits best in these dog groups with the given weighting:")
    for i in range(len(groups)):
        print(percents[i], '% group', groups[i])
        print('The dogs of this group are:', breed_clusters.get(groups[i]))

### Tests

In [115]:
data = breed_data.drop(columns=['Breeds', 'scaled_pop'])
estimate_dog_breed(breed_data, data.iloc[0]) # Affenpinscher
estimate_dog_breed_pop(breed_data, data.iloc[0]) # Affenpinscher
estimate_dog_breed(breed_data, data.iloc[1]) # Afghan Hound
estimate_dog_breed_pop(breed_data, data.iloc[1]) # Afghan Hound
estimate_dog_breed(breed_data, data.iloc[2]) # Airedale Terrier
estimate_dog_breed_pop(breed_data, data.iloc[2]) # Airedale Terrier

We estimate your dog to be:
100.0 %  Havanese
We estimate your dog to be:
100.0 %  Havanese
You're dog fits best in these dog groups with the given weighting:
100.0 % group 10
The dogs of this group are: ['American Bulldog', 'Bichon Frise', 'Bulldog', 'French Bulldog', 'Havanese']
We estimate your dog to be:
100.0 %  Afghan Hound
We estimate your dog to be:
51.0 %  Rottweiler
49.0 %  Yorkshire Terrier
You're dog fits best in these dog groups with the given weighting:
100.0 % group 12
The dogs of this group are: ['Afghan Hound', 'Basset Hound', 'Plott Hound', 'Rhodesian Ridgeback', 'Saluki']
We estimate your dog to be:
100.0 %  Airedale Terrier
We estimate your dog to be:
100.0 %  English Springer Spaniel
You're dog fits best in these dog groups with the given weighting:
48.0 % group 13
The dogs of this group are: ['Poodle (Miniature)', 'Poodle (Standard)', 'Poodle (Toy)']
52.0 % group 53
The dogs of this group are: ['Brittany', 'German Shorthaired Pointer', 'Leonberger', 'Pointer', 'We

### Taking User inputs

In [118]:
age = int(input('How old is your dog in earth years?'))
if age <= 3:
    print("Please understand that your dog is still young and may not have yet developed all of the attributes of it's breed or breed mixture. This makes it far more difficult to guess it's breed / breed mixture.")

How old is your dog in earth years? 10


In [119]:
height = int(input('What height is your dog (cm)?'))
scaled_height = height / max_height

What height is your dog (cm)? 30


In [120]:
weight = int(input('What weight is your dog (kg)?'))
scaled_weight = weight / max_weight

What weight is your dog (kg)? 10


In [122]:
groomingvals = [i for i in df['grooming_frequency_value'].dropna().unique()]
groomingfreqs = [i for i in df['grooming_frequency_category'].dropna().unique()]
for i in groomingfreqs:
    print(groomingfreqs.index(i), i)
groomingval = groomingvals[int(input('From the above unordered list, enter the number that corresponds to the option that best describes your dogs grooming demands.'))]

0 2-3 Times a Week Brushing
1 Daily Brushing
2 Occasional Bath/Brush
3 Weekly Brushing
4 Specialty/Professional


From the above unordered list, enter the number that corresponds to the option that best describes your dogs grooming demands. 2


In [123]:
sheddingvals = [i for i in df['shedding_value'].dropna().unique()]
sheddingfreqs = [i for i in df['shedding_category'].dropna().unique()]
for i in sheddingfreqs:
    print(sheddingfreqs.index(i), i)
sheddingval = sheddingvals[int(input("From the above unordered list, enter the number that corresponds to the option that best describes how frequently your dog sheds it's fur."))]

0 Seasonal
1 Infrequent
2 Occasional
3 Regularly
4 Frequent


From the above unordered list, enter the number that corresponds to the option that best describes how frequently your dog sheds it's fur. 0


In [124]:
energyvals = [i for i in df['energy_level_value'].dropna().unique()]
energycategs = [i for i in df['energy_level_category'].dropna().unique()]
for i in energycategs:
    print(energycategs.index(i), i)
energyval = energyvals[int(input("From the above unordered list, enter the number that corresponds to the option that best describes your dog's energy level."))]

0 Regular Exercise
1 Energetic
2 Needs Lots of Activity
3 Couch Potato
4 Calm


From the above unordered list, enter the number that corresponds to the option that best describes your dog's energy level. 4


In [125]:
trainabilityvals = [i for i in df['trainability_value'].dropna().unique()]
trainabilitycategs = [i for i in df['trainability_category'].dropna().unique()]
for i in trainabilitycategs:
    print(trainabilitycategs.index(i), i)
trainabilityval = trainabilityvals[int(input("From the above unordered list, enter the number that corresponds to the option that best describes how easy your dog is to train."))]

0 Easy Training
1 May be Stubborn
2 Eager to Please
3 Independent
4 Agreeable


From the above unordered list, enter the number that corresponds to the option that best describes how easy your dog is to train. 4


In [126]:
demeanorvals = [i for i in df['demeanor_value'].dropna().unique()]
demeanorcategs = [i for i in df['demeanor_category'].dropna().unique()]
for i in demeanorcategs:
    print(demeanorcategs.index(i), i)
demeanorval = demeanorvals[int(input("From the above unordered list, enter the number that corresponds to the option that best describes the demeanor of your dog."))]

0 Outgoing
1 Aloof/Wary
2 Friendly
3 Alert/Responsive
4 Reserved with Strangers


From the above unordered list, enter the number that corresponds to the option that best describes the demeanor of your dog. 0


In [127]:
y = [scaled_height,scaled_weight,groomingval,sheddingval,energyval,trainabilityval,demeanorval]
estimate_dog_breed_pop(breed_data, y)

We estimate your dog to be:
100.0 %  French Bulldog
You're dog fits best in these dog groups with the given weighting:
100.0 % group 33
The dogs of this group are: ['Basset Fauve de Bretagne', 'Drever', 'Norrbottenspets', 'Redbone Coonhound']
